In [ ]:
from pathlib import Path
from matplotlib import pyplot as plt
from scipy.spatial.distance import euclidean
import numpy as np
import seaborn as sns
import shutil
import cv2 as cv

In [ ]:
def load_labels(labels, image_shape=None):
    """
    image_shape as in cv2 image
    """
    with open(labels) as f:
        labels = f.read()
    labels = labels.strip().split("\n")
    labels = [line.strip().split() for line in labels]
    segments = []
    for line in labels:
        class_ = int(line[0])
        line = [float(x) for x in line]
        points = []
        for i in range(1, len(line), 2):
            if image_shape:
                points.append([image_shape[1] * line[i], image_shape[0] * line[i + 1]])
            else:
                points.append([line[i], line[i + 1]])
        if image_shape:
            points = np.array(points, int)
        else:
            points = np.array(points)
        segments.append([class_, points])
    return segments

def draw_labels(image, labels):
    if type(image) == str:
        image = cv.imread(image)
    if type(labels) == str:
        labels = load_labels(labels, image.shape)
    for class_, contour in labels:
        image = cv.polylines(image, [contour], True, (255, 255, 255))
        image = cv.putText(image, str(class_), contour[0], cv.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255))
    return image

In [ ]:
# Credit: ChatGPT

def perpendicular_distance(point, start, end):
    """Calculate the perpendicular distance from a point to a line segment."""
    if np.all(start == end):
        return euclidean(point, start)
    
    line_vec = end - start
    point_vec = point - start
    line_len = np.dot(line_vec, line_vec)
    t = max(0, min(1, np.dot(point_vec, line_vec) / line_len))
    projection = start + t * line_vec
    return euclidean(point, projection)

def rdp(contour, epsilon):
    """
    Applies the Ramer-Douglas-Peucker (RDP) algorithm to simplify a contour.
    
    Parameters:
        contour (np.ndarray): A Nx2 array of (x, y) coordinates.
        epsilon (float): Tolerance for point reduction (higher -> more aggressive).
    
    Returns:
        np.ndarray: Simplified contour as a Nx2 array.
    """
    if len(contour) < 3:
        return contour

    # Find the point with the maximum perpendicular distance
    start, end = contour[0], contour[-1]
    distances = np.array([perpendicular_distance(p, start, end) for p in contour[1:-1]])
    
    max_idx = np.argmax(distances)
    max_dist = distances[max_idx]
    
    if max_dist > epsilon:
        max_idx += 1  # Offset for skipping the first point
        # Recursive RDP on both segments
        left = rdp(contour[:max_idx+1], epsilon)
        right = rdp(contour[max_idx:], epsilon)
        return np.vstack((left[:-1], right))
    else:
        return np.array([start, end])

In [ ]:
labels = []
for class_, contour in load_labels("datasets/coco2017/val/labels/000000000632.txt"):
    contour = rdp(contour, 0.05)
    labels.append([class_, contour])
# plt.imshow(draw_labels("datasets/coco2017/val/images/000000000632.jpg", labels))